<a href="https://colab.research.google.com/github/gpdsec/Beer_Review_Prediction/blob/master/Review_Classifier_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the libraries

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
X = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Input/X_beer1.csv').astype(np.float32)
X.columns

Index(['beer/ABV', 'review/appearance', 'review/aroma', 'review/palate',
       'review/taste', 'beer/style_Altbier',
       'beer/style_American Adjunct Lager',
       'beer/style_American Amber / Red Ale',
       'beer/style_American Amber / Red Lager',
       'beer/style_American Barleywine', 'beer/style_American Black Ale',
       'beer/style_American Blonde Ale', 'beer/style_American Brown Ale',
       'beer/style_American Dark Wheat Ale',
       'beer/style_American Double / Imperial IPA',
       'beer/style_American Double / Imperial Pilsner',
       'beer/style_American Double / Imperial Stout',
       'beer/style_American IPA', 'beer/style_American Malt Liquor',
       'beer/style_American Pale Ale (APA)', 'beer/style_American Pale Lager',
       'beer/style_American Pale Wheat Ale', 'beer/style_American Porter',
       'beer/style_American Stout', 'beer/style_American Strong Ale',
       'beer/style_American Wild Ale', 'beer/style_Baltic Porter',
       'beer/style_Belgian Da

In [0]:
y = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Input/y_beer1.csv').astype(np.float32)

In [5]:
X

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,beer/style_Altbier,beer/style_American Adjunct Lager,beer/style_American Amber / Red Ale,beer/style_American Amber / Red Lager,beer/style_American Barleywine,beer/style_American Black Ale,beer/style_American Blonde Ale,beer/style_American Brown Ale,beer/style_American Dark Wheat Ale,beer/style_American Double / Imperial IPA,beer/style_American Double / Imperial Pilsner,beer/style_American Double / Imperial Stout,beer/style_American IPA,beer/style_American Malt Liquor,beer/style_American Pale Ale (APA),beer/style_American Pale Lager,beer/style_American Pale Wheat Ale,beer/style_American Porter,beer/style_American Stout,beer/style_American Strong Ale,beer/style_American Wild Ale,beer/style_Baltic Porter,beer/style_Belgian Dark Ale,beer/style_Belgian IPA,beer/style_Belgian Pale Ale,beer/style_Belgian Strong Dark Ale,beer/style_Belgian Strong Pale Ale,beer/style_Berliner Weissbier,beer/style_BiÃ¨re de Garde,beer/style_Black & Tan,beer/style_Bock,beer/style_Braggot,beer/style_California Common / Steam Beer,beer/style_Chile Beer,beer/style_Cream Ale,...,beer/style_Flanders Oud Bruin,beer/style_Flanders Red Ale,beer/style_Foreign / Export Stout,beer/style_Fruit / Vegetable Beer,beer/style_German Pilsener,beer/style_Hefeweizen,beer/style_Herbed / Spiced Beer,beer/style_Irish Dry Stout,beer/style_Irish Red Ale,beer/style_Keller Bier / Zwickel Bier,beer/style_Kristalweizen,beer/style_KÃ¶lsch,beer/style_Lambic - Fruit,beer/style_Lambic - Unblended,beer/style_Light Lager,beer/style_Low Alcohol Beer,beer/style_Maibock / Helles Bock,beer/style_Milk / Sweet Stout,beer/style_Munich Dunkel Lager,beer/style_Munich Helles Lager,beer/style_MÃ¤rzen / Oktoberfest,beer/style_Oatmeal Stout,beer/style_Old Ale,beer/style_Pumpkin Ale,beer/style_Quadrupel (Quad),beer/style_Rauchbier,beer/style_Russian Imperial Stout,beer/style_Rye Beer,beer/style_Saison / Farmhouse Ale,beer/style_Schwarzbier,beer/style_Scotch Ale / Wee Heavy,beer/style_Scottish Ale,beer/style_Scottish Gruit / Ancient Herbed Ale,beer/style_Smoked Beer,beer/style_Tripel,beer/style_Vienna Lager,beer/style_Weizenbock,beer/style_Wheatwine,beer/style_Winter Warmer,beer/style_Witbier
0,5.00,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11.00,4.0,3.5,3.5,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.70,3.5,4.0,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.40,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.40,4.0,3.0,3.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

### Splitting the dataset into the Training set and Test set

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

Futher spliting Test set into Test set and Validation set

In [0]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.3, random_state = 0)

In [8]:
X_train.shape

(26250, 100)

### Feature Scaling

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

In [10]:
X_val.shape

(3375, 100)

## Part 2 - Building the ANN

I am starting with ANN if it does not worked I will switch to other algorithm an see which works

Defining Callback Parameters

In [0]:
es = EarlyStopping(monitor='loss', patience=3)
filepath="/content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5"
md = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

Defining Important Variable

In [0]:
# Important Variables
epochs = 100
num_classes = 10
batch_size = 1024
input_shape = (100,)
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

### Initializing the ANN

In [13]:
ann = Sequential()
# 1st Dense Layer
ann.add(Dense(units=100, input_shape=input_shape, activation='relu'))
ann.add(Dropout(0.25))

# 2nd Dense Layer
ann.add(Dense(units=50, activation='relu'))
ann.add(Dropout(0.25))

# 3rd Dense Layer
ann.add(Dense(units=25, activation='relu'))
ann.add(Dropout(0.25))

# output layer
ann.add(Dense(units=10, activation='softmax'))

# Compiling the ANN
ann.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Summery of ANN
ann.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2

## Part 3 - Training the ANN

### Training the ANN on the Training set

In [14]:
History = ann.fit(X_train,
                    y_train, 
                    batch_size=batch_size,
                    #steps_per_epoch=2048,
                    epochs = epochs,
                    verbose=2,
                    validation_data = (X_test, y_test),
                    callbacks = [es,md],
                    shuffle= True
                    )

Epoch 1/100

Epoch 00001: loss improved from inf to 2.57470, saving model to /content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5
26/26 - 1s - loss: 2.5747 - accuracy: 0.1134 - val_loss: 2.2851 - val_accuracy: 0.1285
Epoch 2/100

Epoch 00002: loss improved from 2.57470 to 2.39947, saving model to /content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5
26/26 - 0s - loss: 2.3995 - accuracy: 0.1456 - val_loss: 2.1732 - val_accuracy: 0.2248
Epoch 3/100

Epoch 00003: loss improved from 2.39947 to 2.28371, saving model to /content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5
26/26 - 0s - loss: 2.2837 - accuracy: 0.1851 - val_loss: 2.0980 - val_accuracy: 0.2804
Epoch 4/100

Epoch 00004: loss improved from 2.28371 to 2.20547, saving model to /content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5
26/26 - 0s - loss: 2.2055 - accuracy: 0.2117 - val_loss: 2.0385 - val_accuracy: 0.3186
Epoch 5/100

Epoch 00005: loss improved from 2.20547 to 2.14974, saving model to /content/drive/

## Part 4 - Making the predictions and evaluating the model

### Predicting the Test set results

In [15]:
ann2 = load_model("/content/drive/My Drive/Colab Notebooks/Model/bestmodel.h5")
# Summery of ANN2
ann2.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2

### Making the Confusion Matrix

As it seems ANN is falled to provide Nessary solution

In [16]:
y_pred = ann2.predict_classes(X_val)
y_pred = y_pred*0.5
print(y_pred)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[3.5 3.  3.  ... 2.5 4.  3.5]


In [17]:
y_val.columns

Index(['review/overall_0.0', 'review/overall_1.0', 'review/overall_1.5',
       'review/overall_2.0', 'review/overall_2.5', 'review/overall_3.0',
       'review/overall_3.5', 'review/overall_4.0', 'review/overall_4.5',
       'review/overall_5.0'],
      dtype='object')

In [0]:
y_val.set_index('review/overall_0.0', inplace=True)


In [0]:
Y_vall =y_val[y_val==1].stack().reset_index().drop(0,1)

In [23]:
y_val.columns

Index(['review/overall_1.0', 'review/overall_1.5', 'review/overall_2.0',
       'review/overall_2.5', 'review/overall_3.0', 'review/overall_3.5',
       'review/overall_4.0', 'review/overall_4.5', 'review/overall_5.0'],
      dtype='object')

In [0]:
y_val = Y_vall.drop(['review/overall_0.0'], axis=1, inplace=False)

In [0]:
def funct(x):
 
  if x == 'review/overall_0.0':
    return 0
  elif x == 'review/overall_1.0':
    return 1
  elif x== 'review/overall_1.5':
    return 1.5
  elif x== 'review/overall_2.0':
    return 2
  elif x== 'review/overall_2.5':
    return 2.5
  elif x== 'review/overall_3.0':
    return 3
  elif x== 'review/overall_3.5':
    return 3.5
  elif x== 'review/overall_4.0':
    return 4
  elif x== 'review/overall_4.5':
    return 4.5
  else:
    return 5


y_val['review'] = y_val.apply(lambda x:funct(x['level_1']),axis=1)

In [44]:
y_val['review']

0       4.5
1       3.0
2       3.5
3       5.0
4       4.0
       ... 
3370    4.0
3371    3.5
3372    3.5
3373    5.0
3374    4.0
Name: review, Length: 3375, dtype: float64

In [52]:
# function to check predicted accuracy
def accuracy(df,k):
  k = 0
  for i in range(len(df)):
    if df[i] == y_pred[i]:
      k = k+1
  
  acc = (k*100)/len(y_val)
  return acc

print(accuracy(y_val['review'], y_pred))


20.94814814814815


In [0]:
# Poor accuracy